In [1]:
# import standard libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import os
import plotly.express as px
import matplotlib

In [2]:
# change default display width of a df
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 2000)

Load data from csv data source and print first 5 rows

In [3]:
df = pd.read_csv('../input/warsaw-pl-flat-prices-sept-2022/Warsaw_flat_prices_25_Sep_22.csv')
print(df.head())

                                        Location       Price  Price per m2  Size M2     Rooms
0                         Warszawa, Praga-Północ  396 000 zł  13 655 zł/m²    29 m²  2 pokoje
1               Warszawa, Bemowo, ul. Secemińska  695 000 zł  11 564 zł/m²  60.1 m²  3 pokoje
2                              Warszawa, Wilanów  828 000 zł  15 055 zł/m²    55 m²  2 pokoje
3                            Warszawa, Białołęka  730 000 zł  13 225 zł/m²  55.2 m²  3 pokoje
4  Warszawa, Śródmieście, ul. Marszałkowska 111a  750 000 zł  19 330 zł/m²  38.8 m²  2 pokoje


Print shape of our data

In [4]:
print("There are {} rows and {} columns".format(df.shape[0], df.shape[1]))

There are 11250 rows and 5 columns


Check if there is any missing data

In [5]:
df.isna().sum()

Location        0
Price           0
Price per m2    0
Size M2         0
Rooms           0
dtype: int64

# Data Cleaning

Create and fill out currency column with USD, PLN or EUR based on the Price column

In [6]:
df.loc[df['Price'].str.contains('$', case=False), 'Currency'] = 'USD'
df.loc[df['Price'].str.contains('zł', case=False), 'Currency'] = 'PLN'
df.loc[df['Price'].str.contains('€', case=False), 'Currency'] = 'EUR'

Check how many listings are in different currencies

In [7]:
print(df['Currency'].value_counts())

PLN    11235
EUR       14
USD        1
Name: Currency, dtype: int64


Clean price column of currency symbols

In [8]:
df.loc[df['Currency'] == 'PLN', 'Price'] = df['Price'].str[:-3]
df.loc[df['Currency'] == 'USD', 'Price'] = df['Price'].str[1:]
df.loc[df['Currency'] == 'EUR', 'Price'] = df['Price'].str[:-2]

Remove space in price column

In [9]:
df['Price'] = df['Price'].str.replace(" ", "")

Remove digits after a comma

In [10]:
df.loc[df['Price'].str.contains(','), 'Price'] = df['Price'].str.split(',').str[0]

Convert price column to integer

In [11]:
df = df.astype({'Price':'int'})

Clean Price per m2 column of currency symbols

In [12]:
df.loc[df['Currency'] == 'PLN', 'Price per m2'] = df['Price per m2'].str[:-6]
df.loc[df['Currency'] == 'USD', 'Price per m2'] = df['Price per m2'].str[1:6]
df.loc[df['Currency'] == 'EUR', 'Price per m2'] = df['Price per m2'].str[:-5]

Remove comma in one USD entry

In [13]:
df.loc[df['Currency'] == 'USD', 'Price per m2'] = df['Price per m2'].str.replace(',', '')

Remove space in Price per m2 column

In [14]:
df['Price per m2'] = df['Price per m2'].str.replace(" ", "")

Remove digits after a comma

In [15]:
df.loc[df['Price per m2'].str.contains(','), 'Price per m2'] = df['Price per m2'].str.split(',').str[0]

Convert price column to integer

In [16]:
df = df.astype({'Price per m2':'int'})

Define a function to get exchange rate of PLN

In [17]:
def get_exchage_rate(curr):
    url = 'https://api.exchangerate.host/latest?base={}&symbols=PLN'.format(curr)
    response = requests.get(url)
    data = response.json()
    rate = data['rates']['PLN']
    return rate

Get exchange rate for USD and EUR

In [18]:
eur_rate = get_exchage_rate('EUR')
usd_rate = get_exchage_rate('USD')

Convert USD and EUR to PLN

In [19]:
df.loc[df['Currency'] == 'EUR', 'Price'] = round(df['Price'] * eur_rate, 0)
df.loc[df['Currency'] == 'USD', 'Price'] = round(df['Price'] * usd_rate, 0)

df.loc[df['Currency'] == 'EUR', 'Price per m2'] = round(df['Price per m2'] * eur_rate, 0)
df.loc[df['Currency'] == 'USD', 'Price per m2'] = round(df['Price per m2'] * usd_rate, 0)

Create district column and fill out with data based on listing location

In [20]:
df.loc[df['Location'].str.contains('Praga-Północ', case=False), 'District'] = 'Praga-Północ'
df.loc[df['Location'].str.contains('Bemowo', case=False), 'District'] = 'Bemowo'
df.loc[df['Location'].str.contains('Wilanów', case=False), 'District'] = 'Wilanów'
df.loc[df['Location'].str.contains('Białołęka', case=False), 'District'] = 'Białołęka'
df.loc[df['Location'].str.contains('Śródmieście', case=False), 'District'] = 'Śródmieście'
df.loc[df['Location'].str.contains('Centrum', case=False), 'District'] = 'Śródmieście'
df.loc[df['Location'].str.contains('Ursynów', case=False), 'District'] = 'Ursynów'
df.loc[df['Location'].str.contains('Mokotów', case=False), 'District'] = 'Mokotów'
df.loc[df['Location'].str.contains('Bielany', case=False), 'District'] = 'Bielany'
df.loc[df['Location'].str.contains('Wola', case=False), 'District'] = 'Wola'
df.loc[df['Location'].str.contains('Ursus', case=False), 'District'] = 'Ursus'
df.loc[df['Location'].str.contains('Targówek', case=False), 'District'] = 'Targówek'
df.loc[df['Location'].str.contains('Ochota', case=False), 'District'] = 'Ochota'
df.loc[df['Location'].str.contains('Praga-Południe', case=False), 'District'] = 'Praga-Południe'
df.loc[df['Location'].str.contains('Żoliborz', case=False), 'District'] = 'Żoliborz'
df.loc[df['Location'].str.contains('Rembertów', case=False), 'District'] = 'Rembertów'
df.loc[df['Location'].str.contains('Włochy', case=False), 'District'] = 'Włochy'
df.loc[df['Location'].str.contains('Wawer', case=False), 'District'] = 'Wawer'
df.loc[df['Location'].str.contains('Wesoła', case=False), 'District'] = 'Wesoła'

Check how many listings were not mapped to district

In [21]:
df['District'].isna().sum()

81

Clean Size M2 column and convert to integer

In [22]:
df['Size M2'] = df['Size M2'].str[:-3]
df['Size M2'] = df['Size M2'].str.strip()
df = df.astype({'Size M2':'float'})

Clean and convert room column to integer

In [23]:
df['Rooms'].unique()

array(['2 pokoje', '3 pokoje', '4 pokoje', '6 pokoi', '1 pokój',
       '5 pokoi', '7 pokoi', '8 pokoi', '10 pokoi', '10+ pokoi'],
      dtype=object)

In [24]:
df['Rooms_int'] = df['Rooms'].str[0]
df.loc[df['Rooms'].str.contains('10', case=False), 'Rooms_int'] = 10
df = df.astype({'Rooms_int':'int'})

Filter out flat prices less than 100k as these listings are probably for rent, not sale

In [25]:
df = df.loc[(df['Price'] > 100000)]

Filter out prices less than 2k per m2

In [26]:
df = df.loc[(df['Price per m2'] > 2000)]

# Exploratory Data Analysis

Check how many listings are in each district

In [27]:
df['District'].value_counts()

Mokotów           1730
Wola              1180
Śródmieście       1152
Praga-Południe    1079
Białołęka          784
Ursynów            741
Bielany            685
Bemowo             594
Ochota             580
Targówek           521
Wilanów            465
Żoliborz           399
Ursus              391
Praga-Północ       295
Włochy             254
Wawer              166
Wesoła              84
Rembertów           65
Name: District, dtype: int64

Show summary statistics

In [28]:
df['Price per m2'].describe().round(2)

count     11246.00
mean      14493.80
std        5344.44
min        2813.00
25%       11393.00
50%       13404.00
75%       16179.00
max      148889.00
Name: Price per m2, dtype: float64

In [29]:
df['Price'].describe().round(2)

count       11246.00
mean       955121.63
std       1044565.40
min        105500.00
25%        550000.00
50%        695000.00
75%        958800.00
max      21600000.00
Name: Price, dtype: float64

In [30]:
df['Size M2'].describe().round(2)

count    11246.00
mean        62.63
std         36.51
min         10.00
25%         43.00
50%         54.00
75%         69.69
max        560.00
Name: Size M2, dtype: float64

See the average m2 price per district

In [31]:
df.groupby('District')['Price per m2'].mean().sort_values(ascending=False).round(2)

District
Śródmieście       21558.25
Żoliborz          16434.23
Wilanów           16024.86
Mokotów           15785.73
Wola              15661.31
Ochota            14558.69
Praga-Północ      13646.36
Ursynów           13166.49
Praga-Południe    12988.45
Włochy            12906.57
Bielany           12881.18
Bemowo            12188.80
Targówek          11365.02
Ursus             11263.68
Białołęka         10492.42
Wesoła            10263.74
Wawer             10041.81
Rembertów          9921.55
Name: Price per m2, dtype: float64

See median m2 price per district

In [32]:
df.groupby('District')['Price per m2'].median().sort_values(ascending=False).round(2)

District
Śródmieście       19086.5
Wilanów           15746.0
Żoliborz          15592.0
Wola              15000.0
Mokotów           14562.0
Ochota            13953.0
Praga-Północ      13200.0
Włochy            12871.0
Ursynów           12863.0
Praga-Południe    12553.0
Bielany           12298.0
Bemowo            12011.0
Targówek          11261.0
Ursus             11128.0
Białołęka         10352.5
Rembertów          9756.0
Wesoła             9748.0
Wawer              9682.0
Name: Price per m2, dtype: float64

See median listing price per district

In [33]:
df.groupby('District')['Price'].median().sort_values(ascending=False).round(2)

District
Wilanów           1049000.0
Śródmieście       1000000.0
Żoliborz           753900.0
Mokotów            747720.0
Ursynów            740000.0
Bemowo             700000.0
Wola               699000.0
Wesoła             692250.0
Ochota             690000.0
Wawer              675500.0
Włochy             652500.0
Praga-Południe     649000.0
Bielany            600000.0
Rembertów          595000.0
Ursus              577000.0
Targówek           575000.0
Białołęka          554000.0
Praga-Północ       550000.0
Name: Price, dtype: float64

See mean listing price per district

In [34]:
df.groupby('District')['Price'].mean().sort_values(ascending=False).round(2)

District
Śródmieście       1717425.57
Wilanów           1317307.55
Żoliborz          1206971.58
Mokotów           1199772.20
Wola               883795.50
Ochota             859391.64
Ursynów            854681.30
Bemowo             776112.12
Wawer              764846.50
Praga-Południe     725650.34
Włochy             725398.32
Wesoła             713366.33
Bielany            687090.21
Praga-Północ       682545.89
Targówek           620478.77
Rembertów          614186.92
Ursus              611283.71
Białołęka          592323.17
Name: Price, dtype: float64

See median size of apartment per district

In [35]:
df.groupby('District')['Size M2'].median().sort_values(ascending=False).round(2)

District
Wesoła            69.00
Wilanów           68.00
Wawer             68.00
Ursynów           62.30
Bemowo            61.25
Rembertów         59.80
Mokotów           55.35
Białołęka         55.00
Śródmieście       54.72
Włochy            54.00
Targówek          54.00
Praga-Południe    53.00
Ursus             52.90
Żoliborz          50.19
Bielany           49.30
Wola              49.00
Ochota            49.00
Praga-Północ      44.99
Name: Size M2, dtype: float64

See how many rooms the listings have

In [36]:
df['Rooms_int'].value_counts()

2     4365
3     4188
4     1207
1     1045
5      325
6       76
7       30
8        5
10       5
Name: Rooms_int, dtype: int64

Plot histogram of Price per m2 column

In [37]:
px.histogram(df, x="Price per m2").show()

Plot histogram with no extreme values & colored by district

In [38]:
px.histogram(df.loc[(df['Price per m2'] < 40000)], x="Price per m2", color="District").show()

Explore outliers

In [39]:
px.histogram(df.loc[(df['Price per m2'] > 40000)], x="Price per m2", color="District").show()

Plot price histogram

In [40]:
px.histogram(df, x="Price").show()

Plot price histogram with listings below 3 mln PLN

In [41]:
px.histogram(df.loc[(df['Price'] < 3000000)], x="Price", color="District").show()

Explore outliers

In [42]:
px.histogram(df.loc[(df['Price'] > 3000000)], x="Price", color="District").show()

Plot scatterplot with m2 and listing price

In [43]:
px.scatter(df, x='Price', y='Size M2', color='District').show()

Plot histogram with number of rooms

In [44]:
px.histogram(df, x="Rooms_int", color="District").show()

# MapBox Chart

Create a dataframe with median price per district

In [45]:
grouped_price = df.groupby('District')['Price'].median().sort_values(ascending=False)
df_grouped_price = pd.DataFrame(grouped_price)
df_grouped_price.reset_index(inplace=True)
df_grouped_price

,District,Price
0,Wilanów,1049000.0
1,Śródmieście,1000000.0
2,Żoliborz,753900.0
3,Mokotów,747720.0
4,Ursynów,740000.0
5,Bemowo,700000.0
6,Wola,699000.0
7,Wesoła,692250.0
8,Ochota,690000.0
9,Wawer,675500.0


Create a dataframe with median price per m2 per district

In [46]:
grouped_pricem2 = df.groupby('District')['Price per m2'].median().sort_values(ascending=False)
df_grouped_pricem2 = pd.DataFrame(grouped_pricem2)
df_grouped_pricem2.reset_index(inplace=True)
df_grouped_pricem2

,District,Price per m2
0,Śródmieście,19086.5
1,Wilanów,15746.0
2,Żoliborz,15592.0
3,Wola,15000.0
4,Mokotów,14562.0
5,Ochota,13953.0
6,Praga-Północ,13200.0
7,Włochy,12871.0
8,Ursynów,12863.0
9,Praga-Południe,12553.0


Merge two dataframes created above

In [47]:
merged = df_grouped_price.merge(df_grouped_pricem2, on='District', how='left')
merged.head()

,District,Price,Price per m2
0,Wilanów,1049000.0,15746.0
1,Śródmieście,1000000.0,19086.5
2,Żoliborz,753900.0,15592.0
3,Mokotów,747720.0,14562.0
4,Ursynów,740000.0,12863.0


Merge dataframe with geo-lat-lon csv to get longitute and latitude data for each district

In [48]:
geo = pd.read_csv('../input/geo-lat-lon/geo.csv')
merged_geo = merged.merge(geo, on='District', how='left')
merged_geo = merged_geo.rename(columns={'Price': 'Median Price', 'Price per m2': 'Median Price per m2'})
merged_geo.head()

,District,Median Price,Median Price per m2,Lat,Lon
0,Wilanów,1049000.0,15746.0,52.164620,21.082968
1,Śródmieście,1000000.0,19086.5,52.228133,21.011287
2,Żoliborz,753900.0,15592.0,52.270040,20.983630
3,Mokotów,747720.0,14562.0,52.208701,21.015364
4,Ursynów,740000.0,12863.0,52.149079,21.045985


Plot mapbox

In [49]:
fig = px.scatter_mapbox(merged_geo, lat='Lat', lon='Lon',title='Median m2 price per district: Sept 2022', hover_name="District", color='Median Price per m2', hover_data=["Median Price", "Median Price per m2"],
                        size="Median Price per m2", zoom=9.5, color_continuous_scale=px.colors.diverging.Geyser)
fig.update_layout(mapbox_style="carto-positron", autosize=True)
fig.show()

# Conclusion

Mokotów, Wola, Śródmieście and Praga-Południe are districts with the most number of listings.
Most of the listings are for 2 and 3 room apartments. 
Śródmieście, Wilanów, Żoliborz  and Mokotów are the most expensive districts both by total price and price per m2.

Price per m2 is heavily right skewed due to a few outliers with extreme values reaching 100k+ per m2. However, most of the listings are priced between 8k and 20k per m2. Median price per m2 is 13,404 PLN.
Price per m2 is dependent on listing location with notably more expensive listings located in Śródmieście, Mokotów and Wilanow


Price is also heavily skewed to the right with extreme values reaching 20+ mln. Most of the listings are priced between 350k and 1 mln PLN.
Median listing price in Warsaw is 695,000 PLN.
Most of the listing above 3 mln are located in Śródmieście.

The average size of an apartment is 62 m2 with the largest apartments located in Wesoła, Wilanów and Wawer, and the smallest in Praga-Północ.